In [1]:
import pandas as pd
from services.jira_service import JiraService  # seu service acima
from datetime import datetime
from services.jira_parser import parse_issues_to_dataframe
import json


jira = JiraService()
board_id = 71  # atualize se necessário

backlog_data = jira.get_raw_backlog_issues(board_id).get("issues", [])
backlog_df = parse_issues_to_dataframe(backlog_data)




🔄 Coletando JSON completo do backlog (board 71)...
✅ Total de 106 issues coletadas do backlog.


In [2]:
print(len(backlog_data))

106


In [12]:
for issue in backlog_data:
    fields = issue.get("fields", {})
    if issue.get("key") in ["SQD-690"]:
        Épico = fields.get("epic", {}).get("summary") or "Não informado"
        print(Épico)    








CNAB LP
